# Решение задачи:

## Комментарий по поводу структуры Google документа

В составлении текста для двух Google документов помогал **ChatGPT**. Ему была представлена подробная информация по нейро-специалистам, расписаны их задачи и услуги, а также описана их целевая группа. Также при сооставлении промпта была определена важность выполнения моего запроса и расписана основная идея, для которой и нужно было выполнить поставленную мной задачу.

Далее я попросил его структурировать документ, опираясь на представленный мной шаблон:
```
Заголовок 1 уровня (логическое описание, тема к которой относиться фрагмент)
Заголовок 2 уровня (отражает смысл фрагмента или группы, в которую входит фрагмент)
Фрагмент (из первоначального текста, либо оптимизированный chatGPT)
```

Далее я скопировал полученный результат и донастроил его в Google документе, установив в необходимых местах формат *"Заголовок 1 уровня"* и "Заголовок 2 уровня".

Также в инструкции для нейро-сотрудника (в Google документе) и промпте я решил описать желаемую структуру, которая должна быть на входе языковой модели, входными данными которой является Google документ.

## Основная часть кода

Сначала необходимо установить библиотеки для работы с API OpenAI, векторную базу данных ChromaDB, фреймворк LangChain для работы с LLM, библиотеку tiktoken для подсчета токенов, а также Gradio для построения интерфейса.

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.1 MB/s eta 0:00:0

Импортируем все необходимые библиотеки:

In [ ]:
from langchain.docstore.document import Document #работа с документами в langchain
from langchain.embeddings.openai import OpenAIEmbeddings #эмбеддинги для OpenAI
from langchain.vectorstores import Chroma #доступ к векторной базе данных
from langchain.text_splitter import CharacterTextSplitter #разделение текста на куски или чанки (chunk)
import requests #отправка запросов
from openai import OpenAI #доступ к OpenAI
import gradio as gr #отрисовка интерфейса с помощью grad
import tiktoken #библиотека подсчёта токенов
                #без запроcов к OpenAI, тем самым не тратим деньги на запросы
import re #для работы с регулярными выражениями
import getpass #для работы с паролями
import os #для работы с окружением и файловой системой

Напишем код для запроса ввода ключа от OpenAI:

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

Введите OpenAI API Key:··········


Любую разработку необходимо начинать с проектирования, с построения ее архитектуры. И делать это необходимо исходя из поставленной задачи (технического задания).

Так давайте поставим перед собой следующее *техническое задание*:
1. В качестве интерфейса используем **Gradio**;
2. В интерфейсе должны быть заложены 2 механизма: обучение и отправка запросов нейро-сотруднику;
3. Реализовать предзаполненный набор нейро-сотрудников, чтобы пользователь мог выбирать, с кем происходит общение. При выборе нейро-сотрудника показать его промпт и возможный вопрос, дать возможность их редактировать;
4. Предоставить пользователю доступ к исходным данным для обучения в виде ссылок на гугл-документы;
5. Взаимодействие между векторной базой, LLM и пользователем организовать на фреймворке **LangChain**.

Учитывая техническое задание, мы должны создать список поддерживаемых нейро-сотрудников и задать им предварительные настройки в виде: промпта, примера пользовательского запроса, с ссылкой на обучающие наборы данных.

Эти данные мы выносим в отдельную переменную. А ещё лучше вынести в отдельный файл, создав файл конфигурации. Такой подход широко используется в программировании для разделения данных и кода, что позволяет изменять данные (создавать новых нейро-сотрудников), без необходимости править код:

In [ ]:
models = [
              {
                "doc": "https://docs.google.com/document/d/1eUsc_fIU5BwDkNH0akiAFf3CxpWW3K3DMnYbS1HEjLw/edit",
                "prompt": '''Ты специалист технической поддержки в интернет-магазине, к тебе могут обращаться гости магазина и его сотрудники за подсказками, ответами на их вопросы или для решения их проблем.
                        Компания является интернет-магазином.
                        Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у тех, кто тебе обратился, больше не осталось вопросов к тебе.
                        Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
                        Отвечай максимально точно по документу, не придумывай ничего от себя, будь вежлив и старайся войти в положении того, кто к тебе обратился.
                        На входе языковой модели подаются фрагменты из векторной базы-данных в виде:
                        - Заголовок 1 уровня: общая тема фрагмента.
                        - Заголовок 2 уровня: смысл фрагмента.
                        - Фрагмент: исходный текст.
                        Данные для векторной базы данных бери из этого документа: ''',
                "name": "Нейро-специалист технической поддержки интернет-магазина",
                "query": "Напиши примеры приветствий и прощаний"
              },
              {
                "doc": "https://docs.google.com/document/d/1_oTGHq7DB4o9zzW16D_3XmrCgXbt_bT46Ii-nVzgFCA/edit",
                "prompt": '''Ты специалист технической части интернет-магазина, к тебе будут обращаться весь технический отдел интернет-магазина за подсказками и ответами на их вопросы.
                        Компания является интернет-магазином.
                        Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у тех, кто тебе обратился, больше не осталось вопросов к тебе.
                        Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
                        Отвечай максимально точно по документу, не придумывай ничего от себя, будь вежлив и старайся войти в положении того, кто к тебе обратился.
                        На входе языковой модели подаются фрагменты из векторной базы-данных в виде:
                        - Заголовок 1 уровня: общая тема фрагмента.
                        - Заголовок 2 уровня: смысл фрагмента.
                        - Фрагмент: исходный текст.
                        Данные для векторной базы данных бери из этого документа: ''',
                "name": "Нейро-специалист технической части интернет-магазина",
                "query": "Напиши основные характеристики платформы"
              }
            ]

Этот код реализует класс *GPT* для создания AI-ассистента, работающего с документами. Вот что происходит:

1. **Инициализация**: При создании объекта инициализируются атрибуты для логов, модели ИИ и векторной базы знаний, а также устанавливается подключение к OpenAI.

2. **Загрузка данных**: Метод `load_search_indexes` извлекает текст из Google Docs по URL, валидирует ссылку и передает данные на обработку.

3. **Обработка документов**:

*   Текст разбивается на фрагменты (**chunks**) размером 1024 символа;
*   Рассчитывается количество токенов через `tiktoken`;
*   Создаются векторные представления текста через `OpenAIEmbeddings`;
*   Данные сохраняются в векторную БД **Chroma**.

4. Работа с запросами:

*   При запросе (`answer_index`) система ищет 5 наиболее релевантных фрагментов из БД;
*   Формирует контекстное сообщение с промптом и найденными фрагментами;
*   Отправляет запрос в **OpenAI**, контролируя расход токенов;
*   Возвращает ответ модели с детальной статистикой использования токенов.

Класс реализует **RAG-архитектуру** (**Retrieval-Augmented Generation**), сочетая поиск по базе знаний с генерацией ответов языковой моделью, что позволяет работать с большими документами.

In [ ]:
#объявляем класс нейро-сотрудника
class GPT():
    #объявляем конструктор класса, для передачи имени модели и инициализации атрибутов класса
    def __init__(self, model="gpt-3.5-turbo"):
        self.log = ''               #атрибут для сбора логов (сообщений)
        self.model = model          #атрибут для хранения выбранной модели OpenAI
        self.search_index = None    #атрибут для хранения ссылки на базу знаний (если None, то модель не обучена)
        self.api_key = os.environ["OPENAI_API_KEY"]
        self.base_url = "https://api.vsegpt.ru/v1"
        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url) #при инициализации запрашиваем ключ от OpenAI

    #метод загрузки текстового документа в векторную базу знаний
    def load_search_indexes(self, url):
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url) #извлекаем document ID гугл документа из URL с помощью регулярных выражений

        #если ID не найден - генерируем исключение
        if match_ is None:
            raise ValueError('Неверный Google Docs URL')

        doc_id = match_.group(1) #первый элемент в результате поиска
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt') #скачиваем гугл документ по его ID в текстовом формате
        response.raise_for_status() #при неудачных статусах запроса будет вызвано исключение
        text = response.text #извлекаем данные как текст

        return self.create_embedding(text) #вызываем метод векторизации текста и сохранения в векторную базу данных

    #подсчет числа токенов в строке по имени модели
    def num_tokens_from_string(self, string):
            """Возвращает число токенов в строке"""
            encoding = tiktoken.encoding_for_model(self.model)  #получаем кодировщик по имени модели
            num_tokens = len(encoding.encode(string))           #расчитываем строку с помощью кодировщика
            return num_tokens                                   #возвращаем число токенов

    #метод разбора текста и его сохранение в векторную базу знаний
    def create_embedding(self, data):
        source_chunks = [] #список документов, полученных из фрагментов текста
        #разделяем текст на строки по \n (перенос на новую строку) или длине фрагмента (chunk_size=1024) с помощью сплитера
        #chunk_overlap=0 - означает, что фрагменты не перекрываются друг с другом.
        #если больше нуля, то захватываем дополнительное число символов от соседних чанков.
        splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

        #применяем splitter (функцию расщепления) к данным и перебираем все получившиеся чанки (фрагменты)
        for chunk in splitter.split_text(data):
            source_chunks.append(Document(page_content=chunk, metadata={})) #LangChain работает с документами, поэтому из текстовых чанков мы создаем фрагменты документов

        count_token = self.num_tokens_from_string(' '.join([x.page_content for x in source_chunks])) #подсчет числа токенов в документах без запроса к OpenAI (экономим денежные средства)
        self.log += f'Количество токенов в документе : {count_token}\n' #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации

        embeddings = OpenAIEmbeddings(openai_api_key=self.api_key, openai_api_base=self.base_url)
        self.search_index = Chroma.from_documents(source_chunks, embeddings) #создание индексов документа. Применяем к нашему списку документов эмбеддингов OpenAi и в таком виде загружаем в базу ChromaDB
        self.log += f'Данные из документа загружены в в векторную базу данных\n' #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации

        return self.search_index #возвращаем ссылку на базу данных

    #демонстрация более аккуратного расчета числа токенов в зависимости от модели
    def num_tokens_from_messages(self, messages, model):
        """Возвращает число токенов из списка сообщений"""
        try:
            encoding = tiktoken.encoding_for_model(model) #получаем кодировщик по имени модели
        except KeyError:
            print("Предупреждение: модель не создана. Используйте cl100k_base кодировку.")
            encoding = tiktoken.get_encoding("cl100k_base") # сли по имени не нашли, то используем базовый для моделей OpenAI
        #выбор модели
        if model in {
            "gpt-3.5-turbo-0613",
            "gpt-3.5-turbo-16k-0613",
            "gpt-4-0314",
            "gpt-4-32k-0314",
            "gpt-4-0613",
            "gpt-4-32k-0613",
            "gpt-4o",
            "gpt-4o-2024-05-13"
            }:
            tokens_per_message = 3 #дополнительное число токенов на сообщение
            tokens_per_name = 1    #токенов на имя
        elif model == "gpt-3.5-turbo-0301":
            tokens_per_message = 4  #каждое сообщение содержит <im_start>{role/name}\n{content}<im_end>\n
            tokens_per_name = -1  #если есть имя, то роль не указывается
        elif "gpt-3.5-turbo" in model:
            self.log += f'Внимание! gpt-3.5-turbo может обновиться в любой момент. Используйте gpt-3.5-turbo-0613. \n'
            return self.num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
        elif "gpt-4" in model:
            self.log += f'Внимание! gpt-4 может обновиться в любой момент. Используйте gpt-4-0613. \n'
            return self.num_tokens_from_messages(messages, model="gpt-4-0613")
        else: #исключение, если модель не поддерживается
            raise NotImplementedError(
                f"""num_tokens_from_messages() не реализован для модели {model}."""
            )

        #запускаем подсчет токенов
        num_tokens = 0                        #счетчик токенов
        for message in messages:              #цикл по всем сообщениям
            num_tokens += tokens_per_message  #прибовляем число токенов на каждое сообщение
            for key, value in message.items():
                num_tokens += len(encoding.encode(value)) #считаем токены в сообщении с помощью кодировщика
                if key == "name":                     #если встретили имя
                    num_tokens += tokens_per_name     #то добавили число токенов на
        num_tokens += 3                               #каждый ответ оборачивается в <|start|>assistant<|message|>
        return num_tokens                             #возвращаем число токенов


    #метод запроса к языковой модели
    def answer_index(self, system, topic, temp = 1):
        #проверяем обучена ли наша модель
        if not self.search_index:
            self.log += 'Модель необходимо обучить! \n'
            return ''

        docs = self.search_index.similarity_search(topic, k=5) #выборка документов по схожести с запросом из векторной базы данных, topic- строка запроса, k - число извлекаемых фрагментов
        self.log += 'Выбираем документы по степени схожести с вопросом из векторной базы данных: \n ' #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации
        message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'Отрывок документа №{i+1}:\n' + doc.page_content + '\\n' for i, doc in enumerate(docs)])) #очищаем запрос от двойных пустых строк. Каждый фрагмент подписываем: Отрывок документа № и дальше порядковый номер
        self.log += f'{message_content} \n' #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации

        #в системную роль помещаем найденные фрагменты и промпт, в пользовательскую - вопрос от пользователя
        messages = [
            {"role": "system", "content": system + f"{message_content}"},
            {"role": "user", "content": topic}
        ]

        self.log += f"\n\nТокенов использовано на вопрос по версии TikToken: {self.num_tokens_from_messages(messages, self.model)}\n" #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации


        #запрос к языковой моделе
        completion = self.client.chat.completions.create(
            model=self.model,   #используемая модель
            messages=messages,  #список форматированных сообщений с ролями
            temperature=temp    #точность ответов модели
        )


        #вместо вывода print, мы формируем переменную log для дальнейшего вывода в gradio информации
        self.log += '\nСтатистика по токенам от языковой модели:\n'
        self.log += f'Токенов использовано всего (вопрос): {completion.usage.prompt_tokens} \n'       #число токенов на вопрос по расчетам LLM
        self.log += f'Токенов использовано всего (вопрос-ответ): {completion.usage.total_tokens} \n'  #число токенов на вопрос и ответ по расчетам LLM

        return completion.choices[0].message.content #возвращаем результат предсказания

Этот код реализует веб-интерфейс с использованием **Gradio** для взаимодействия с нейросетевым ассистентом на базе класса *GPT*. При запуске создается экземпляр класса *GPT* с моделью "`gpt-3.5-turbo`". Интерфейс включает выпадающий список для выбора предопределенных конфигураций данных из переменной models, при изменении которого автоматически обновляются связанные поля: отображается название выбранной конфигурации, системный промпт (с очисткой от лишних пробелов), пример пользовательского запроса и активная HTML-ссылка на обучающий документ *Google Docs*.

Основное взаимодействие происходит через две кнопки: "*Обучить модель*" и "*Запрос к модели*". При нажатии первой кнопки происходит загрузка выбранного документа в векторную базу знаний через метод `load_search_indexes`, а вторая кнопка инициирует запрос к языковой модели через метод `answer_index`, используя введённый промпт и запрос пользователя. Результаты работы системы выводятся в двух соседних текстовых полях: в первом отображается ответ нейросети, во втором - технические логи процесса, включая информацию о токенах и этапах обработки данных. Весь интерфейс запускается через вызов `demo.launch()`:

In [ ]:
gpt = GPT("gpt-3.5-turbo") #объявляем экземпляр класса GPT (созданный ранее) и передаем ему в конструктор модель LLM, с которой будем работать

blocks = gr.Blocks() #Gradio позволяет объединять элементы в блоки

#работаем с блоком
with blocks as demo:
    subject = gr.Dropdown([(elem["name"], index) for index, elem in enumerate(models)], label="Данные") #объявляем элемент выбор из списка (с подписью Данные), список выбирает из поля name нашей переменной models
    name = gr.Label(show_label=False) #здесь отобразится выбранное имя name из списка
    prompt = gr.Textbox(label="Промт", interactive=True) #промпт для запроса к LLM (по умолчанию поле prompt из models)
    link = gr.HTML() #ссылка на файл обучения (по умолчанию поле doc из models)
    query = gr.Textbox(label="Запрос к LLM", interactive=True) #поле пользовательского запроса к LLM (по умолчанию поле query из models)


    #функция на выбор нейро-сотрудника в models
    #ей передается параметр subject - выбранное значение в поле списка
    #а возвращаемые значения извлекаются из models
    def onchange(dropdown):
      return [
          models[dropdown]['name'],                               #имя возвращается без изменения
          re.sub('\t+|\s\s+', ' ', models[dropdown]['prompt']),   #в промте удаляются двойные пробелы \s\s+ и табуляция \t+
          models[dropdown]['query'],                              #запрос возвращается без изменения
          f"<a target='_blank' href = '{models[dropdown]['doc']}'>Документ для обучения</a>" #ссылка на документ оборачивается в html тег <a> (https://htmlbook.ru/html/a)
          ]

    #при изменении значения в поле списка subject, вызывается функция onchange
    #ей передается параметр subject - выбранное значение в поле списка
    #а возвращаемые значения устанавливаются в элементы name, prompt, query и link
    subject.change(onchange, inputs = [subject], outputs = [name, prompt, query, link])

    #строку в gradio можно разделить на столбцы (каждая кнопка в своем столбце)
    with gr.Row():
        train_btn = gr.Button("Обучить модель")       #кнопка запуска обучения
        request_btn = gr.Button("Запрос к модели")    #кнопка отправки запроса к LLM

    def train(dropdown):
        try:
            gpt.load_search_indexes(models[dropdown]['doc'])
            return gpt.log
        except Exception as e:
            gpt.log += f"\nОшибка: {str(e)}"
            return gpt.log

    #вызываем метод запроса к языковой модели из класса GPT
    def predict(p, q):
        result = gpt.answer_index(
            p,
            q
        )
        #возвращает список из ответа от LLM и log от класса GPT
        return [result, gpt.log]

    #выводим поля response с ответом от LLM и log (вывод сообщений работы класса GPT) на 2 колонки
    with gr.Row():
        response = gr.Textbox(label="Ответ LLM") #текстовое поле с ответом от LLM
        log = gr.Textbox(label="Логирование")    #текстовое поле с выводом сообщений от GPT


    #при нажатии на кнопку train_btn запускается функция обучения train_btn с параметром subject
    #результат выполнения функции сохраняем в текстовое поле log - лог выполнения
    train_btn.click(train, [subject], log)

    #при нажатии на кнопку request_btn запускается функция отправки запроса к LLM request_btn с параметром prompt, query
    #результат выполнения функции сохраняем в текстовые поля  response - ответ модели, log - лог выполнения
    request_btn.click(predict, [prompt, query], [response, log])

demo.launch() #запуск приложения

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e8ee5117bf00a2f3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
